In [ ]:
from google.colab import drive
 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! cp /content/drive/My\ Drive/corpus_utils/* ./

In [ ]:
! cp ./*.py /content/drive/My\ Drive/corpus_utils/

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin
from pymystem3 import Mystem

--2020-09-19 17:43:47--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-09-19 17:43:47--  http://cache-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm901.cdn.yandex.net (cache-mskm901.cdn.yandex.net)... 5.45.220.11, 2a02:6b8:0:2002::12
Connecting to cache-mskm901.cdn.yandex.net (cache-mskm901.cdn.yandex.net)|5.45.220.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.3’

mystem-3.0-linux3.1 100%[===================>] 

In [ ]:
from structures import *
from aligner import *
from tokenizer import *


In [ ]:
import glob
from parsing import extract_sentences


norm_sentences = []

for filename in glob.glob('/content/drive/My Drive/corpus_data_json/*/*/*.json'):
    norm_sentences += extract_sentences(filename)

In [ ]:
len(norm_sentences)

63530

In [ ]:
def lemmatize_sentences(sentences: List[NormSentences]) -> List[SentencePair]:
    lemmatized_sentences = []
    mystem_analyzer = Mystem()
    indexes_to_delete = []


    for i, sentence in enumerate(sentences):
        source = list(filter(lambda x : x[0].isalpha() or x[0].isnumeric(),
                                mystem_analyzer.lemmatize(sentence.source)))
        target = list(filter(lambda x : x[0].isalpha() or x[0].isnumeric(),
                                mystem_analyzer.lemmatize(sentence.target)))
        if len(source) == 0 or len(target) == 0:
            indexes_to_delete.append(i)
            continue
        lemmatized_sentences.append(SentencePair(source, target))
    for i in indexes_to_delete[::-1]:
        sentences.pop(i)
    return lemmatized_sentences

In [ ]:
prepared_sentences = lemmatize_sentences(norm_sentences)

In [ ]:
from tokenizer import get_token_to_index, tokenize_sents

t_idx_src, t_idx_tgt = get_token_to_index(prepared_sentences)
tokenized_sentences = tokenize_sents(prepared_sentences,t_idx_src,t_idx_tgt)

In [ ]:
from aligner import WordAligner

word_aligner = WordAligner(len(t_idx_src), len(t_idx_tgt), 20)
word_aligner.fit(tokenized_sentences)
aligned = word_aligner.align(tokenized_sentences)

In [ ]:
import os
import csv
SIZE = 1000

with open('hier.csv', 'w') as csvfile:
    for j in range(0, len(prepared_sentences), SIZE):
        filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(j, min(j + 2, len(prepared_sentences))):
            ru = []
            for idx in aligned[i]:
                ru.append(prepared_sentences[i].source[idx[0] - 1])
            filewriter.writerow([norm_sentences[i].source.replace(',', '，')])
            filewriter.writerow([norm_sentences[i].target.replace(',', '，')])
            filewriter.writerow(prepared_sentences[i].target)
            filewriter.writerow(ru)
            filewriter.writerow([])
!cp hier.csv "drive/My Drive/hieroglyphs/"
os.rename("drive/My Drive/hieroglyphs/hier.csv", "drive/My Drive/hieroglyphs/hierNEW.csv")